# 受講者の演習環境バックアップをリストアします
---

バックアップした受講者の演習環境を展開します。

# バックアップファイルを指定します

リストアするバックアップファルを指定します。
- ホームディレクトリからの相対パスで指定します。
- フォルダを指定した場合、フォルダ内の全ての `*.tgz` ファイルが対象となります。
- バックアップファイルがローカルマシンにある場合は、下記の手順でアップロードできます。
  1. ブラウザのHome画面で、リストア作業用フォルダを開きます。
  2. 画面右上の[Upload]ボタンをクリックします。ファイル選択ダイアログが開きます。
  3. ファイルを選択して[開く]ボタンをクリックします。 バックアップファイル名がリストされます。
  4. バックアップファイル名の行の右端に表示される[Upload]ボタンをクリックします。

**次のセルを実行して、バックアップファイルを設定してください。**

In [ ]:
BACKUP_FILE='backup/0123/student-a01@example.com-backup.tgz'

# バックアップを展開します

- バックアップはファイル名に含まれる名前の受講者のフォルダに展開されます。
- 同一のファイルは上書きされます。

In [ ]:
import os, sys, re, hashlib, string, tempfile, shutil, subprocess

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

try:
    BACKUP_FILE
except NameError:
    BACKUP_FILE=None

if BACKUP_FILE is None:
    print('BACKUP_FILE にバックアップファイルを指定してください', file=sys.stderr)
    SRC_PATH=None
elif os.path.isabs(BACKUP_FILE):
    SRC_PATH=os.path.realpath(BACKUP_FILE)
else:
    SRC_PATH=os.path.realpath(os.path.join(os.getenv('HOME'), BACKUP_FILE))

backup_files=set()
if SRC_PATH is None:
    print('バックアップファイルがありません', file=sys.stderr)
elif os.path.isdir(SRC_PATH):
    for (root, dirs, files) in os.walk(SRC_PATH):
        for file in files:
            path = os.path.join(root, file)
            if path.endswith('.tgz') or path.endswith('.tar.gz'):
                backup_files.add(os.path.realpath(path))
elif os.path.exists(SRC_PATH):
    print('mode SINGLE FILE')
    backup_files.add(os.path.realpath(SRC_PATH))
else:
    print('{}: バックアップファイルが見つかりません'.format(BACKUP_FILE), file=sys.stderr)

#
with tempfile.TemporaryDirectory() as tmp_dir:
    for tgzfile in backup_files:
        basename = os.path.basename(tgzfile)
        m = re.compile(r'^(.+)-backup\.(tgz|tar\.gz)$').match(basename)
        user_mail = m.group(1) if (m is not None) else None
        if (user_mail is None) or (len(user_mail) < 1):
            print('WARNING: 受講者名が不明のファイル: {}'.format(tgzfile))
            continue
        print('受講者[{}]のバックアップを展開します'.format(user_mail))
        user_name = get_username_from_mail_address(user_mail)
        user_dir_info = os.stat(os.path.join('/home/jupyter/workspace', user_name))
        user_uid = user_dir_info.st_uid
        user_gid = user_dir_info.st_gid
        user_tmp_dir = '{}/{}'.format(tmp_dir, user_name)
        os.makedirs(user_tmp_dir, exist_ok=True)
        try:
            tarcmd = 'tar -xz -C "{}" -f "{}"'.format(user_tmp_dir, tgzfile)
            cp = subprocess.run(tarcmd, shell=True, cwd=user_tmp_dir, stderr=subprocess.PIPE)
            for line in cp.stderr.splitlines():
                print('tar> {}'.format(line.strip().decode('utf-8')), file=sys.stderr)
            if cp.returncode != 0:
                print('ERROR: tar コマンドが失敗しました', file=sys.stderr)
                continue
        except OSError as e:
            print('ERROR: tar コマンドが実行できません：{}'.format(e), file=sys.stderr)
            continue
        user_dir = '/home/jupyter/workspace/{}'.format(user_name)
        try:
            cmd = 'cd {}'.format(tmp_dir) \
                + ' && sudo -n -s mkdir -p "{}"'.format(user_dir) \
                + ' && sudo -n -s rsync -vhi -rLkKEtW "{}/" "{}/"'.format(user_tmp_dir, user_dir) \
                + ' && sudo -n -s chown -R {}:{} "{}/"'.format(user_uid, user_gid, user_dir) \
                + ' && sudo -n -s find "{}/" -name ".*" -prune'.format(user_dir) \
                                     + ' -o -type d -execdir chmod 0755 "{}" "+"' \
                + ' && sudo -n -s find "{}/" -name ".*" -prune'.format(user_dir) \
                                     + ' -o -type f -execdir chmod 0644 "{}" "+"' \
                + ' && true'
            cp = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            for line in cp.stdout.splitlines():
                print('rsync> {}'.format(line.strip().decode('utf-8')))
            for line in cp.stderr.splitlines():
                print('rsync> {}'.format(line.strip().decode('utf-8')), file=sys.stderr)
            if cp.returncode == 0:
                print('-> 完了')
            else:
                print('ERROR: rsync コマンドが失敗しました', file=sys.stderr)
                continue
        except OSError as e:
            print(type(e))
            print('ERROR: rsync コマンドが実行できません：{}'.format(e), file=sys.stderr)
            continue